In [1]:
import pandas as pd
import numpy as np

In [2]:
start_date = "1960-01-01"
end_date = "2024-12-16"

In [3]:
import pandas_datareader as pdr

In [4]:
factors_ff3_monthly_raw = pdr.DataReader(
name="F-F_Research_Data_Factors",
data_source="famafrench",
start=start_date,
end=end_date)[0]
factors_ff3_monthly = (factors_ff3_monthly_raw
.divide(100)
.reset_index(names="month")
.assign(month=lambda x: pd.to_datetime(x["month"].astype(str)))
.rename(str.lower, axis="columns")
.rename(columns={"mkt-rf": "mkt_excess"})
)

<ipython-input-4-2e8d585653d5>:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  factors_ff3_monthly_raw = pdr.DataReader(
<ipython-input-4-2e8d585653d5>:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  factors_ff3_monthly_raw = pdr.DataReader(


In [5]:
factors_ff5_monthly_raw = pdr.DataReader(
name="F-F_Research_Data_5_Factors_2x3",
data_source="famafrench",
start=start_date,
end=end_date)[0]
factors_ff5_monthly = (factors_ff5_monthly_raw
.divide(100)
.reset_index(names="month")
.assign(month=lambda x: pd.to_datetime(x["month"].astype(str)))
.rename(str.lower, axis="columns")
.rename(columns={"mkt-rf": "mkt_excess"})
)

<ipython-input-5-6fa4eb749b87>:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  factors_ff5_monthly_raw = pdr.DataReader(
<ipython-input-5-6fa4eb749b87>:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  factors_ff5_monthly_raw = pdr.DataReader(


In [6]:
factors_ff3_daily_raw = pdr.DataReader(
name="F-F_Research_Data_Factors_daily",
data_source="famafrench",
start=start_date,
end=end_date)[0]
factors_ff3_daily = (factors_ff3_daily_raw
.divide(100)
.reset_index(names="date")
.rename(str.lower, axis="columns")
.rename(columns={"mkt-rf": "mkt_excess"})
)

<ipython-input-6-d7271dea11ae>:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  factors_ff3_daily_raw = pdr.DataReader(


In [7]:
industries_ff_monthly_raw = pdr.DataReader(
name="10_Industry_Portfolios",
data_source="famafrench",
start=start_date,
end=end_date)[0]
industries_ff_monthly = (industries_ff_monthly_raw
.divide(100)
.reset_index(names="month")
.assign(month=lambda x: pd.to_datetime(x["month"].astype(str)))
.rename(str.lower, axis="columns")
)

<ipython-input-7-a7e322091467>:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  industries_ff_monthly_raw = pdr.DataReader(
<ipython-input-7-a7e322091467>:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  industries_ff_monthly_raw = pdr.DataReader(
<ipython-input-7-a7e322091467>:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  industries_ff_monthly_raw = pdr.DataReader(
<ipython-input-7-a7e322091467>:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead

In [13]:
factors_q_monthly_link = (
    "https://global-q.org/uploads/1/2/2/6/122679606/"
    "q5_factors_monthly_2022.csv"
)

factors_q_monthly = (
    pd.read_csv(factors_q_monthly_link)
    .assign(
        month=lambda x: pd.to_datetime(
            x["year"].astype(str) + "-"
            + x["month"].astype(str) + "-01"
        )
    )
    .drop(columns=["R_F", "R_MKT", "year"])
    .rename(columns=lambda x: x.replace("R_", "").lower())
    .query(f"month >= '{start_date}' and month <= '{end_date}'")
    .assign(**{col: lambda x: x[col] / 100 for col in ["me", "ia", "roe", "eg"]})
)



In [14]:
sheet_id = "1g4LOaRj4TvwJr9RIaA_nwrXXWTOy46bP"
sheet_name = "macro_predictors.xlsx"
macro_predictors_link = (
f"https://docs.google.com/spreadsheets/d/{sheet_id}"
f"/gviz/tq?tqx=out:csv&sheet={sheet_name}"
)

In [15]:
macro_predictors = (
pd.read_csv(macro_predictors_link, thousands=",")
.assign(
month=lambda x: pd.to_datetime(x["yyyymm"], format="%Y%m"),
dp=lambda x: np.log(x["D12"])-np.log(x["Index"]),
dy=lambda x: np.log(x["D12"])-np.log(x["D12"].shift(1)),
ep=lambda x: np.log(x["E12"])-np.log(x["Index"]),
de=lambda x: np.log(x["D12"])-np.log(x["E12"]),
tms=lambda x: x["lty"]-x["tbl"],
dfy=lambda x: x["BAA"]-x["AAA"]
)
.rename(columns={"b/m": "bm"})
.get(["month", "dp", "dy", "ep", "de", "svar", "bm",
"ntis", "tbl", "lty", "ltr", "tms", "dfy", "infl"])
.query("month >= @start_date and month <= @end_date")
.dropna()
)

In [16]:
cpi_monthly = (pdr.DataReader(
name="CPIAUCNS",
data_source="fred",
start=start_date,
end=end_date
)
.reset_index(names="month")
.rename(columns={"CPIAUCNS": "cpi"})
.assign(cpi=lambda x: x["cpi"]/x["cpi"].iloc[-1])
)

In [17]:
#import sqlite3

In [18]:
#tidy_finance = sqlite3.connect(database="data/tidy_finance_python.sqlite")

OperationalError: unable to open database file

In [21]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [22]:
import sqlite3

# Path to the database file in Google Drive
db_path = "/content/drive/My Drive/tidy_finance_python.sqlite"

# Connect to the database
tidy_finance = sqlite3.connect(db_path)


In [23]:
(factors_ff3_monthly
.to_sql(name="factors_ff3_monthly",
con=tidy_finance,
if_exists="replace",
index=False)
)

756

In [24]:
pd.read_sql_query(
sql="SELECT month, rf FROM factors_ff3_monthly",
con=tidy_finance,
parse_dates={"month"}
)

,month,rf
0,1960-01-01,0.0033
1,1960-02-01,0.0029
2,1960-03-01,0.0035
3,1960-04-01,0.0019
4,1960-05-01,0.0027
...,...,...
751,2022-08-01,0.0019
752,2022-09-01,0.0019
753,2022-10-01,0.0023
754,2022-11-01,0.0029


In [27]:
data_dict = {
    "factors_ff5_monthly": factors_ff5_monthly,
    "factors_ff3_daily": factors_ff3_daily,
    "industries_ff_monthly": industries_ff_monthly,
    "factors_q_monthly": factors_q_monthly,
    "macro_predictors": macro_predictors,
    "cpi_monthly": cpi_monthly
}

for key, value in data_dict.items():
    value.to_sql(
        name=key,
        con=tidy_finance,
        if_exists="replace",
        index=False
    )


In [28]:
#import pandas as pd
#import sqlite3

In [31]:
#Use the code in the next line

#tidy_finance = sqlite3.connect(database="/content/drive/My Drive/tidy_finance_python.sqlite")
#factors_q_monthly = pd.read_sql_query(
#sql="SELECT * FROM factors_q_monthly",
#con=tidy_finance,
#parse_dates={"month"}
#)

In [32]:
import sqlite3
import pandas as pd

# Connect to the database in Google Drive
tidy_finance = sqlite3.connect(database="/content/drive/My Drive/tidy_finance_python.sqlite")

# Query the factors_q_monthly table
factors_q_monthly = pd.read_sql_query(
    sql="SELECT * FROM factors_q_monthly",
    con=tidy_finance,
    parse_dates=["month"]  # Corrected to a list
)


In [39]:
factors_q_monthly = pd.read_sql_query(
    sql="SELECT * FROM factors_q_monthly",
    con=tidy_finance,
    parse_dates=["month"]  # Corrected to a list
)

In [34]:
#check if table exists
cursor = tidy_finance.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in the database:", tables)


Tables in the database: [('factors_ff3_monthly',), ('factors_ff5_monthly',), ('factors_ff3_daily',), ('industries_ff_monthly',), ('factors_q_monthly',), ('macro_predictors',), ('cpi_monthly',)]


In [35]:
import os

# Check if the file exists at the specified path
db_path = "/content/drive/My Drive/tidy_finance_python.sqlite"
print("Database file exists:", os.path.exists(db_path))

# Attempt to connect
try:
    tidy_finance = sqlite3.connect(db_path)
    print("Connection successful!")
except Exception as e:
    print("Connection failed:", e)


Database file exists: True
Connection successful!


In [36]:
cursor = tidy_finance.cursor()

# List all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in the database:", tables)


Tables in the database: [('factors_ff3_monthly',), ('factors_ff5_monthly',), ('factors_ff3_daily',), ('industries_ff_monthly',), ('factors_q_monthly',), ('macro_predictors',), ('cpi_monthly',)]


In [37]:
try:
    cursor.execute("SELECT COUNT(*) FROM factors_q_monthly")
    row_count = cursor.fetchone()[0]
    print(f"Number of rows in factors_q_monthly: {row_count}")
except Exception as e:
    print("Error querying the table:", e)


Number of rows in factors_q_monthly: 672


In [38]:
try:
    cursor.execute("SELECT * FROM factors_q_monthly LIMIT 5")
    rows = cursor.fetchall()
    for row in rows:
        print(row)
except Exception as e:
    print("Error executing query:", e)


('1967-01-01 00:00:00', -0.021768999999999997, -0.021768999999999997, -0.021768999999999997, -0.021768999999999997)
('1967-02-01 00:00:00', 0.022210999999999998, 0.022210999999999998, 0.022210999999999998, 0.022210999999999998)
('1967-03-01 00:00:00', -0.010360000000000001, -0.010360000000000001, -0.010360000000000001, -0.010360000000000001)
('1967-04-01 00:00:00', -0.017259, -0.017259, -0.017259, -0.017259)
('1967-05-01 00:00:00', 0.001578, 0.001578, 0.001578, 0.001578)
